<a href="https://colab.research.google.com/github/granantuin/LEVX_class/blob/master/four_class_LEVX_dir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
def evaluate(y_test,y_pred):
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred)),"// Model Accuracy=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()),"// Model precision=28%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean()),"// Model recall=28%")
  results= precision_recall_fscore_support(y_test, y_pred, average='weighted', )
  print("Precision weighted=","{:.2%}".format(results[0]),"//Model weighted=56%")
  print("Recall weighted =","{:.2%}".format(results[1]),"//Model weighted=53%")
  print("****************")
  print(df)

In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
drive_metar="/content/drive/My Drive/Colab Notebooks/model_vs_data/Metar_2018/"
dr=[drive_4km,drive_1km,drive_metar]
y_data=pd.read_excel(dr[2]+"y_LEVX_dir.xlsx",index_col=0)
x_data=pd.read_csv(dr[2]+"x_LEVX.csv",index_col=0)
x_data=x_data.iloc[:,0:9]
y_data=y_data[y_data>0]# delete station errors
result = x_data.join(y_data, how='outer').dropna()
x_data=result.iloc[:,0:9]
y_data=result.iloc[:,9:10]

In [0]:
y_data

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_index()

**Dummy **

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.3, random_state=5)
y_pred = DummyClassifier(strategy="most_frequent").fit(x_train, y_train).predict(x_test)


In [40]:
evaluate(y_test,y_pred)

    E  N  NE  NW    S  SE  SW  W
E   0  0   0   0   44   0   0  0
N   0  0   0   0  329   0   0  0
NE  0  0   0   0  211   0   0  0
NW  0  0   0   0   83   0   0  0
S   0  0   0   0  509   0   0  0
SE  0  0   0   0    6   0   0  0
SW  0  0   0   0  423   0   0  0
W   0  0   0   0  407   0   0  0
****************
Accuracy= 25.30% // Model Accuracy=45%
Average precision = 3.16% // Model precision=28%
Average recall = 12.50% // Model recall=28%
Precision weighted= 6.40% //Model weighted=56%
Recall weighted = 25.30% //Model weighted=53%
****************
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.000000     0.0  0.000000
NW      0.000000     0.0  0.000000
S       0.252982     1.0  0.403808
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifiers**

In [42]:
clas = [RandomForestClassifier(n_estimators= 200, max_depth= 100, bootstrap= True),
       ExtraTreesClassifier(),AdaBoostClassifier(),GaussianNB()]


x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.3, random_state=5)
y_pred = clas[0].fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E    N  NE  NW    S  SE   SW    W
E   3    6  12   1    8   1    4    9
N   1  209  44  13    6   0   22   34
NE  4  107  56   3    5   0   12   24
NW  0   25   5  13    4   0    5   31
S   3   15   4   0  338   0  114   35
SE  2    1   0   0    2   0    0    1
SW  3   20   7   4  123   0  223   43
W   1   40   7  12   23   0   52  272
****************
Accuracy= 55.37% // Model Accuracy=45%
Average precision = 39.43% // Model precision=28%
Average recall = 37.31% // Model recall=28%
Precision weighted= 53.89% //Model weighted=56%
Recall weighted = 55.37% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.176471  0.068182  0.098361
N       0.494090  0.635258  0.555851
NE      0.414815  0.265403  0.323699
NW      0.282609  0.156627  0.201550
S       0.664047  0.664047  0.664047
SE      0.000000  0.000000  0.000000
SW      0.516204  0.527187  0.521637
W       0.605791  0.668305  0.635514


**Synthetic samples**

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data["label"], test_size=0.4, random_state=27)
print("Original train samples=",x_train.shape[0])
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train)
print("Synthetic train samples=",x_train.shape[0])

y_pred = clas[0].fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

Original train samples= 4023
Synthetic train samples= 8304
     E    N   NE  NW    S  SE   SW    W
E   21    3   13   0    2   9    3    6
N   17  216  102  41    3   1   22   37
NE  25   88   99  21    3   4   13   26
NW   5   29    6  39    2   2    4   37
S   29    8   10   9  441  20  166   26
SE   5    2    2   0    3   1    2    1
SW  23   19   12  10  123  20  296   39
W   20   29   21  55   23   8   57  304
****************
Accuracy= 52.81% // Model Accuracy=45%
Average precision = 40.05% // Model precision=28%
Average recall = 41.86% // Model recall=28%
Precision weighted= 56.55% //Model weighted=56%
Recall weighted = 52.81% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.144828  0.368421  0.207921
N       0.548223  0.492027  0.518607
NE      0.373585  0.354839  0.363971
NW      0.222857  0.314516  0.260870
S       0.735000  0.622003  0.673797
SE      0.015385  0.062500  0.024691
SW      0.525755  0.5461

**K_ folds**

In [0]:
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy","precision_weighted","recall_weighted","f1_weighted"]
scores = cross_validate(clas[0], x_data, y_data["label"], scoring=scoring,
                        cv=5, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1 : {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))
print("Recall weighted: {:.2%} (+/- {:.2%})" .format (scores["test_recall_weighted"].mean(), scores["test_recall_weighted"].std() * 2))
print("Precision weighted: {:.2%} (+/- {:.2%})" .format (scores["test_precision_weighted"].mean(), scores["test_precision_weighted"].std() * 2))
print("f1 weighted: {:.2%} (+/-{:.2%} )".format (scores["test_f1_weighted"].mean(), scores["test_f1_weighted"].std() * 2))

**Tuning RandomForest**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data["label"], test_size=0.25,
                                                    random_state=27)
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(x_train, y_train)
print("best parameters",rf_random.best_params_)

**K folds**

In [0]:
clas=RandomForestClassifier(n_estimators= 200, min_samples_split= 10, min_samples_leaf= 2, max_features= 'sqrt', max_depth= 50, bootstrap= True)
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy","precision_weighted","recall_weighted","f1_weighted"]
scores = cross_validate(clas, x_data, y_data["label"], scoring=scoring,
                        cv=5, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1 : {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))
print("Recall weighted: {:.2%} (+/- {:.2%})" .format (scores["test_recall_weighted"].mean(), scores["test_recall_weighted"].std() * 2))
print("Precision weighted: {:.2%} (+/- {:.2%})" .format (scores["test_precision_weighted"].mean(), scores["test_precision_weighted"].std() * 2))
print("f1 weighted: {:.2%} (+/-{:.2%} )".format (scores["test_f1_weighted"].mean(), scores["test_f1_weighted"].std() * 2))